In [35]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import hashlib

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [36]:
hash_object = hashlib.md5(b'Hello World')
hash_object.hexdigest()

b10a8db164e0754105b7a99be72e3fe5


In [29]:
from preprocessing.preprocessing import *

In [30]:
def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

In [31]:
tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [34]:
hash('data/v1.0_sample_nq-train-sample.jsonl')

-603305005776901102

In [32]:
import json_lines
import os
data = []
with open('data/v1.0_sample_nq-train-sample.jsonl', 'rb') as f: # opening file in binary(rb) mode    
   for item in json_lines.reader(f):
       data.append(item) #or use print(item['X']) for printing specific data

In [33]:
len(data)

200

In [20]:
################################################

In [21]:
dt = data[-110]
dt.keys()

dict_keys(['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens'])

In [22]:
dt['question_text']

'who is the actor that plays lucifer on tv'

In [23]:
#dt['document_tokens'][:10]

In [24]:
dt['annotations']

[{'annotation_id': 4568212507333827460,
  'long_answer': {'candidate_index': 9,
   'end_byte': 58641,
   'end_token': 183,
   'start_byte': 58114,
   'start_token': 130},
  'short_answers': [{'end_byte': 58137,
    'end_token': 134,
    'start_byte': 58120,
    'start_token': 131}],
  'yes_no_answer': 'NONE'}]

In [25]:
outputs = convert_examples_to_features(dt, tokenizer, 512)

target byte range:TargetByteRange(short_start=58120, short_end=58137, long_start=58114, long_end=58641)
DocSpan(start=0, length=500)
span start byte: 92, span end byte: 64207
short span
s: 99
e: 101
DocSpan(start=500, length=500)
span start byte: 64266, span end byte: 75795
DocSpan(start=1000, length=500)
span start byte: 75795, span end byte: 100463
DocSpan(start=1500, length=67)
span start byte: 100677, span end byte: 105274


In [26]:
# answer
if outputs:
    print(outputs[0].targets)

[(99, 101)]


In [27]:
{i: t for i, t in enumerate(outputs[0].tokens) if i >= outputs[0].targets[0][0] and i <= outputs[0].targets[0][1]}

{99: 'thomas', 100: 'john', 101: 'ellis'}

In [39]:
short_answer_start = dt['annotations'][0]['short_answers'][0]['start_byte']
short_answer_end = dt['annotations'][0]['short_answers'][0]['end_byte']
[t for t in dt['document_tokens'] if t['start_byte'] >= short_answer_start and t['end_byte'] <= short_answer_end]

[{'end_byte': 58126,
  'html_token': False,
  'start_byte': 58120,
  'token': 'Thomas'},
 {'end_byte': 58131,
  'html_token': False,
  'start_byte': 58127,
  'token': 'John'},
 {'end_byte': 58137,
  'html_token': False,
  'start_byte': 58132,
  'token': 'Ellis'}]

In [40]:
dt['annotations']

[{'annotation_id': 4568212507333827460,
  'long_answer': {'candidate_index': 9,
   'end_byte': 58641,
   'end_token': 183,
   'start_byte': 58114,
   'start_token': 130},
  'short_answers': [{'end_byte': 58137,
    'end_token': 134,
    'start_byte': 58120,
    'start_token': 131}],
  'yes_no_answer': 'NONE'}]

In [22]:
long_answer_start = dt['annotations'][0]['long_answer']['start_byte']
long_answer_end = dt['annotations'][0]['long_answer']['end_byte']
end = dt['annotations'][0]['long_answer']['end_token']+1
start = dt['annotations'][0]['long_answer']['start_token']
dt['document_tokens'][start:end]

[{'end_byte': 60884, 'html_token': True, 'start_byte': 60881, 'token': '<P>'},
 {'end_byte': 60887, 'html_token': False, 'start_byte': 60884, 'token': 'AMC'},
 {'end_byte': 60897,
  'html_token': False,
  'start_byte': 60888,
  'token': 'announced'},
 {'end_byte': 60901, 'html_token': False, 'start_byte': 60898, 'token': 'the'},
 {'end_byte': 60908,
  'html_token': False,
  'start_byte': 60902,
  'token': 'series'},
 {'end_byte': 60913, 'html_token': False, 'start_byte': 60908, 'token': "'"},
 {'end_byte': 60926,
  'html_token': False,
  'start_byte': 60914,
  'token': 'cancellation'},
 {'end_byte': 60929, 'html_token': False, 'start_byte': 60927, 'token': 'in'},
 {'end_byte': 60934,
  'html_token': False,
  'start_byte': 60930,
  'token': 'July'},
 {'end_byte': 60939,
  'html_token': False,
  'start_byte': 60935,
  'token': '2012'},
 {'end_byte': 60940, 'html_token': False, 'start_byte': 60939, 'token': ','},
 {'end_byte': 60944, 'html_token': False, 'start_byte': 60941, 'token': 'but

In [5]:
dt = data[0]

In [6]:
dt.keys()

dict_keys(['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens'])

In [84]:
dt['document_tokens'][:100]

[{'end_byte': 98, 'html_token': False, 'start_byte': 92, 'token': 'Church'},
 {'end_byte': 101, 'html_token': False, 'start_byte': 99, 'token': 'of'},
 {'end_byte': 109, 'html_token': False, 'start_byte': 102, 'token': 'England'},
 {'end_byte': 111, 'html_token': False, 'start_byte': 110, 'token': '-'},
 {'end_byte': 121,
  'html_token': False,
  'start_byte': 112,
  'token': 'wikipedia'},
 {'end_byte': 58127, 'html_token': True, 'start_byte': 58074, 'token': '<H1>'},
 {'end_byte': 58133,
  'html_token': False,
  'start_byte': 58127,
  'token': 'Church'},
 {'end_byte': 58136, 'html_token': False, 'start_byte': 58134, 'token': 'of'},
 {'end_byte': 58144,
  'html_token': False,
  'start_byte': 58137,
  'token': 'England'},
 {'end_byte': 58149,
  'html_token': True,
  'start_byte': 58144,
  'token': '</H1>'},
 {'end_byte': 58643, 'html_token': False, 'start_byte': 58637, 'token': '``'},
 {'end_byte': 58644, 'html_token': False, 'start_byte': 58643, 'token': 'C'},
 {'end_byte': 58647, 'htm

In [15]:
#set([t['token'] for dt in data for t in dt['document_tokens'] if t['html_token']])

In [14]:
from IPython.core.display import display, HTML
display(HTML(dt['document_html']))

In [54]:
#dt['long_answer_candidates']

In [55]:
dt['question_tokens']

['when',
 'is',
 'the',
 'last',
 'episode',
 'of',
 'season',
 '8',
 'of',
 'the',
 'walking',
 'dead']

In [24]:
dt['annotations'][0]

{'annotation_id': 6782080525527814293,
 'long_answer': {'candidate_index': 92,
  'end_byte': 96948,
  'end_token': 3538,
  'start_byte': 82798,
  'start_token': 2114},
 'short_answers': [{'end_byte': 96731,
   'end_token': 3525,
   'start_byte': 96715,
   'start_token': 3521}],
 'yes_no_answer': 'NONE'}

In [26]:
end_byte_ix = dt['annotations'][0]['short_answers'][0]['end_byte']
start_byte_ix = dt['annotations'][0]['short_answers'][0]['start_byte']
start_byte_ix, end_byte_ix

(96715, 96731)

In [13]:
tokens = dt['document_tokens']

In [31]:
[token for token in tokens if token['start_byte'] >= start_byte_ix and token['end_byte'] <= end_byte_ix]

[{'end_byte': 96720,
  'html_token': False,
  'start_byte': 96715,
  'token': 'March'},
 {'end_byte': 96724, 'html_token': False, 'start_byte': 96722, 'token': '18'},
 {'end_byte': 96725, 'html_token': False, 'start_byte': 96724, 'token': ','},
 {'end_byte': 96731,
  'html_token': False,
  'start_byte': 96727,
  'token': '2018'}]

In [36]:
end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
start_byte_ix_long, end_byte_ix_long

(82798, 96948)

In [8]:
#' '.join([token['token'] for token in tokens if token['start_byte'] >= start_byte_ix_long and token['end_byte'] <= end_byte_ix_long and not token['html_token']])

In [10]:
long_contains_short = 0
short = []
only_long = []
nb_short_answers = []
yes_no_answers = []
multiple_short_answers_q = []
for dt in data:
    if dt['annotations'][0]['yes_no_answer'] != 'NONE':
        yes_no_answers.append(dt)
    short_answers = dt['annotations'][0]['short_answers']
    if short_answers:
        if len(short_answers) > 1:
            multiple_short_answers_q.append(dt)
        
        end_byte_ix = short_answers[0]['end_byte']
        start_byte_ix = short_answers[0]['start_byte']
        end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
        start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
        short.append(dt)  
    else:
        only_long.append(dt) 
print(len(only_long))
print(len(short))
print(len(multiple_short_answers_q))
print(len(yes_no_answers))

130
70
1
1


In [46]:
yes_no_answers[0]['annotations']

[{'annotation_id': 12820076856447911203,
  'long_answer': {'candidate_index': 56,
   'end_byte': 73401,
   'end_token': 1931,
   'start_byte': 72467,
   'start_token': 1864},
  'short_answers': [],
  'yes_no_answer': 'YES'}]

In [36]:
multiple_short_answers_q[0]['annotations'][0]['short_answers']

[{'end_byte': 61401,
  'end_token': 457,
  'start_byte': 61397,
  'start_token': 456},
 {'end_byte': 61653,
  'end_token': 471,
  'start_byte': 61647,
  'start_token': 470},
 {'end_byte': 61913,
  'end_token': 491,
  'start_byte': 61899,
  'start_token': 489}]

In [11]:
only_long[0]['annotations']

[{'annotation_id': 13676402902866580638,
  'long_answer': {'candidate_index': -1,
   'end_byte': -1,
   'end_token': -1,
   'start_byte': -1,
   'start_token': -1},
  'short_answers': [],
  'yes_no_answer': 'NONE'}]

In [13]:

#display(HTML(multiple_short_answers_q['document_html']))

In [89]:
dt['long_answer_candidates'][24]

{'end_byte': 75688,
 'end_token': 501,
 'start_byte': 73956,
 'start_token': 412,
 'top_level': True}

In [72]:
dt = only_long[1]
end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
dt['annotations']

[{'annotation_id': 7446307064203576492,
  'long_answer': {'candidate_index': 0,
   'end_byte': 49862,
   'end_token': 473,
   'start_byte': 42864,
   'start_token': 30},
  'short_answers': [],
  'yes_no_answer': 'NONE'}]

In [73]:
tokens = dt['document_tokens']
' '.join([token['token'] for token in tokens if token['start_byte'] >= start_byte_ix_long and token['end_byte'] <= end_byte_ix_long and not token['html_token']])


"The Man in the High Castle Genre Alternate history Drama Dystopia Thriller Science fiction Created by Frank Spotnitz Based on The Man in the High Castle by Philip K. Dick Starring Alexa Davalos Rupert Evans Luke Kleintank DJ Qualls Joel de la Fuente Cary - Hiroyuki Tagawa Rufus Sewell Brennan Brown Callum Keith Rennie Bella Heathcote Opening theme `` Edelweiss '' , performed by Jeanette Olsson Composer ( s ) Henry Jackman Dominic Lewis Country of origin United States Original language ( s ) English No. of seasons No. of episodes 20 ( list of episodes ) Production Executive producer ( s ) Ridley Scott Frank Spotnitz Christian Baute Isa Dick Hackett Stewart Mackinnon Christopher Tricarico Producer ( s ) Michael Cedar Jean Higgins Jordan Sheehan David W. Zucker Location ( s ) Seattle , Washington Monroe , Washington Vancouver , British Columbia Cinematography James Hawkinson Gonzalo Amat Editor ( s ) Kathrynn Himoff Running time 48 -- 60 minutes Production company ( s ) Amazon Studios Sc

In [74]:
print(dt['question_text'])
#display(HTML(dt['document_html']))

how many season of the man in the high castle
